<a href="https://colab.research.google.com/github/Rad-Monk/TF-Notebooks/blob/main/in_context_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets pdfminer.six torch


In [ ]:
from pdfminer.high_level import extract_text

def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

pdf_text = extract_text_from_pdf('/content/7. DS GRAPH.pdf')
with open('document.txt', 'w') as f:
    f.write(pdf_text)


In [ ]:
pdf_text

In [ ]:
import re

def clean_text_preserve_paragraphs(text):
    # Split the text into paragraphs based on double newlines
    paragraphs = text.split('\n\n')

    # Clean each paragraph separately
    cleaned_paragraphs = []
    for paragraph in paragraphs:
        # Replace multiple spaces within a paragraph with a single space
        cleaned_paragraph = re.sub(r'\s+', ' ', paragraph).strip()
        cleaned_paragraphs.append(cleaned_paragraph)

    # Join the cleaned paragraphs back together, preserving the double newline separation
    return '\n\n'.join(cleaned_paragraphs)

# Example usage:
pdf_text = """This is an example paragraph. It may contain multiple spaces.

This is another paragraph.    With inconsistent spacing."""

cleaned_text = clean_text_preserve_paragraphs(pdf_text)
print(cleaned_text)


In [ ]:
# def split_into_paragraphs(text):
#     return text.split('\n\n')

# paragraphs = split_into_paragraphs(cleaned_text)


In [ ]:
# len(paragraphs)

In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def chunk_text(paragraphs, max_length=512):
    chunks = []
    current_chunk = []
    current_length = 0
    for paragraph in paragraphs:
        paragraph_length = len(tokenizer.encode(paragraph, add_special_tokens=False))
        if current_length + paragraph_length > max_length:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0
        current_chunk.append(paragraph)
        current_length += paragraph_length
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

text_chunks = chunk_text(paragraphs)


In [ ]:
text_chunks

In [ ]:
from transformers import pipeline

qa_pipeline = pipeline('question-answering', model='distilbert-base-uncased', tokenizer=tokenizer)


In [ ]:
len(text_chunks)

In [ ]:
def find_answer(question, text_chunks):
    best_answer = {"score": 0, "answer": ""}
    for chunk in text_chunks:
        result = qa_pipeline({
            'question': question,
            'context': text_chunks[1]
        })
        if result['score'] > best_answer['score']:
            best_answer = result
    return best_answer['answer']

question = "What is the main topic of the document?"
answer = find_answer(question, text_chunks)
print(answer)
